# Perspectives on COVID-19

## Configuring Libraries for the Almond Kernel

First, we'll make a bintray repository with libraries available to your almond kernel.

In [202]:
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")

interp.repositories() ++= Seq(myBT)

myBT: coursierapi.MavenRepository = MavenRepository(https://dl.bintray.com/neelsmith/maven)

Next, we bring in specific libraries from the new repository using almond's `$ivy` magic:

In [203]:

import $ivy.`org.plotly-scala::plotly-almond:0.7.1`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// if you want to have the plots available without an internet connection:
// init(offline=true)

// restrict the output height to avoid scrolling in output cells
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

import $ivy.$                                      

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// if you want to have the plots available without an internet connection:
// init(offline=true)

// restrict the output height to avoid scrolling in output cells


## Imports

From this point on, your notebook consists of completely generic Scala, with the CITE Libraries available to use.

In [204]:
import almond.display.UpdatableDisplay
import almond.interpreter.api.DisplayData.ContentType
import almond.interpreter.api.{DisplayData, OutputHandler}

import java.io.File
import java.io.PrintWriter

import scala.io.Source

import java.text.SimpleDateFormat
import java.util.Date


import almond.display.UpdatableDisplay

import almond.interpreter.api.DisplayData.ContentType

import almond.interpreter.api.{DisplayData, OutputHandler}


import java.io.File

import java.io.PrintWriter


import scala.io.Source


import java.text.SimpleDateFormat

import java.util.Date


## Useful Functions

Save a string to a names file:

In [205]:
def saveString(s:String, filePath:String = "", fileName:String = "temp.txt"):Unit = {
		 val writer = new PrintWriter(new File(s"${filePath}${fileName}"))
         writer.write(s)
         writer.close()
	}

defined function saveString

Like `.split`, but preserving the character we split on:

In [206]:
def splitWithSplitter(text: String, puncs: String): Vector[String] = {
	//val regexWithSplitter = s"((?<=${puncs})|(?=${puncs}))"
    val regexWithSplitter = s"((?<=${puncs}))"
	text.split(regexWithSplitter).toVector.filter(_.size > 0)
}

defined function splitWithSplitter

Pretty Print Things:

In [207]:
def showMe(v:Any):Unit = {
  v match {
    case _:Vector[Any] => println(s"""\n----\n${v.asInstanceOf[Vector[Any]].mkString("\n")}\n----\n""")
    case _:Iterable[Any] => println(s"""\n----\n${v.asInstanceOf[Iterable[Any]].mkString("\n")}\n----\n""")
    case _ => println(s"\n-----\n${v}\n----\n")
  }
}

defined function showMe

## Load Some Data

Load up-to-date data from the NY Times. Source: <https://github.com/nytimes/covid-19-data>.

In [208]:
val dataLines: Vector[String] = {
    scala.io.Source.fromURL("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv").mkString.split("\n").toVector
}

// quick test
val badLines = dataLines.filter( l => {
    l.split(",").size != 5
})

assert ( badLines.size == 0 )

dataLines: Vector[String] = Vector(
  "date,state,fips,cases,deaths",
...
badLines: Vector[String] = Vector()

## Comparative Data

Source for USA Data: [“Deaths and Mortality”, CDC](https://www.cdc.gov/nchs/fastats/deaths.htm).

Source for South Carolina Data: [“Stats of the State of South Carolina”, CDC](https://www.cdc.gov/nchs/pressroom/states/southcarolina/southcarolina.htm)

### USA

Source for USA Data: [“Deaths and Mortality”, CDC](https://www.cdc.gov/nchs/fastats/deaths.htm).

In [209]:
val usa_2017_Total_Deaths: Int = 2813502

// Chronic lower respiratory diseases
val usa_2017_Total_Deaths_CLRD: Int = 160201

// Influenza and Pneumonia
val usa_2017_Total_Deaths_IPn: Int = 55672

// All respiratory-related deaths
val usa_2017_Total_Deaths_Resp: Int = usa_2017_Total_Deaths_CLRD + usa_2017_Total_Deaths_IPn

// Suicide
val usa_2017_Suicide_Total: Int = 47173

// Heart Disease
val usa_2017_Heart_Total: Int = 647457

// Accidents
val usa_2017_Accident_Total: Int = 169936

// Cancer
val usa_2017_Cancer_Total: Int = 599108



usa_2017_Total_Deaths: Int = 2813502
usa_2017_Total_Deaths_CLRD: Int = 160201
usa_2017_Total_Deaths_IPn: Int = 55672
usa_2017_Total_Deaths_Resp: Int = 215873
usa_2017_Suicide_Total: Int = 47173
usa_2017_Heart_Total: Int = 647457
usa_2017_Accident_Total: Int = 169936
usa_2017_Cancer_Total: Int = 599108

### South Carolina

Source for 2018 South Carolina Data: [“Stats of the State of South Carolina”, CDC](https://www.cdc.gov/nchs/pressroom/states/southcarolina/southcarolina.htm) and [SC DHEC](https://www.scdhec.gov/vital-records/parentage/sc-vital-records-data-and-statistics)

In [210]:
// Total Deaths
val sc_2018_Total_Deaths: Int = 50633

// Chronic Lower Respiratory Disease
val sc_2017_Total_Deaths_CLRD: Int = 2990

// Influenza and Pneumonia
val sc_2017_Total_Deaths_IPn: Int = 882

// All Respiratory-Related Deaths
val sc_2017_Daily_Deaths_All_Resp = sc_2017_Daily_Deaths_CLRD + sc_2017_Daily_Deaths_IPn

sc_2018_Total_Deaths: Int = 50633
sc_2017_Total_Deaths_CLRD: Int = 2990
sc_2017_Total_Deaths_IPn: Int = 882
sc_2017_Daily_Deaths_All_Resp: Int = 10

### England & Wales

Source for Data:[The Office of National Statistics, UK](https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/datasets/weeklyprovisionalfiguresondeathsregisteredinenglandandwales)


In [211]:
val uk_weeks = Vector("3-Jan-20","10-Jan-20","17-Jan-20","24-Jan-20","31-Jan-20","7-Feb-20","14-Feb-20","21-Feb-20","28-Feb-20","6-Mar-20","13-Mar-20","20-Mar-20","27-Mar-20")
val uk_Death = Vector(12254,14058,12990,11856,11612,10986,10944,10841,10816,10895,11019,10645,11141)
val uk_Deaths_5yr_Ave = Vector(12175,13822,13216,12760,12206,11925,11627,11548,11183,11498,11205,10573,10130)
val uk_2018_Resp_Deaths = Vector(2361,3075,2829,2672,2479,2197,2081,2128,1900,2302,2271,1946,1657)
val uk_2020_Resp_Deaths = Vector(2141,2477,2188,1893,1746,1572,1602,1619,1546,1557,1488,1514,1534)
val uk_Covid_Deaths = Vector(0,0,0,0,0,0,0,0,0,0,5,103,539)

// A little testing
assert(
    (uk_weeks.size == uk_Death.size) &&
    (uk_weeks.size == uk_Deaths_5yr_Ave.size) &&
    (uk_weeks.size == uk_2018_Resp_Deaths.size) &&
    (uk_weeks.size == uk_2020_Resp_Deaths.size) &&
    (uk_weeks.size == uk_Covid_Deaths.size)
)

uk_weeks: Vector[String] = Vector(
  "3-Jan-20",
...
uk_Death: Vector[Int] = Vector(
  12254,
...
uk_Deaths_5yr_Ave: Vector[Int] = Vector(
  12175,
...
uk_2018_Resp_Deaths: Vector[Int] = Vector(
  2361,
...
uk_2020_Resp_Deaths: Vector[Int] = Vector(
  2141,
...
uk_Covid_Deaths: Vector[Int] = Vector(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 103, 539)

## Make Data Structures

For State data-points from the NY Times:

In [212]:
case class StateDatum( date: String, state: String, cases: Int, deaths: Int)

defined class StateDatum

For Aggregate data (just a Vector of the above):

In [213]:
case class RunningTally( days: Vector[StateDatum] )

defined class RunningTally

A daily snapshot (for the USA or a single state)

In [214]:
case class DailySnapshot( date: String, newCases: Int, newDeaths: Int, totalCases: Int, totalDeaths: Int)

defined class DailySnapshot

## Load Data

In [215]:

val vst: Vector[StateDatum] = dataLines.tail.map( dl => {
    val fields: Vector[String] = dl.split(",").toVector
    val date: String = fields(0)
    val state: String = fields(1)
    val cases: Int = fields(3).toInt
    val deaths: Int = fields(4).toInt
    StateDatum(date, state, cases, deaths)
})

val rt = RunningTally(vst)


vst: Vector[StateDatum] = Vector(
  StateDatum("2020-01-21", "Washington", 1, 0),
...
rt: RunningTally = RunningTally(
  Vector(
...

## Data Functions

Taking, by default, our running-tally data (`rt`) and an optional `Option[String]` that can specify a state (defaults to `None`), return a `Vector[DailySnapshot]`, which will include new cases, new deaths, total cases, and total deaths.

In [216]:
def totalNewDaily(data: Vector[StateDatum] = rt.days, state: Option[String] = None): Vector[DailySnapshot] = {
    val sortedData: Vector[StateDatum] = {
        val filtered: Vector[StateDatum] = {
            state match {
                case Some(s) => data.filter(_.state == s)
                case None => {
                    val groupedByDay: Vector[(String, Vector[StateDatum])] = {
                        data.groupBy(_.date).toVector
                    }
                    val merged: Vector[StateDatum] = groupedByDay.map( gbd => {
                        val vec = gbd._2
                        val vecDate: String = vec.head.date
                        val vecState: String = "USA"
                        val vecCases: Int = vec.map(_.cases).sum
                        val vecDeaths: Int = vec.map(_.deaths).sum
                        StateDatum(vecDate, vecState, vecCases, vecDeaths)
                    })
                    merged
                }
            }
        }
        val sorted: Vector[StateDatum] = filtered.sortBy(_.date)
        sorted
    }
    // We don't want a running, cumulative tally, but new cases/deaths each day
    sortedData.zipWithIndex.map( sd => {
        val d: StateDatum = sd._1
        val i: Int = sd._2
        val newCases: Int = {
            if (i == 0) d.cases
            else {
                val totalToday: Int = d.cases
                val totalPrev: Int = {
                    sortedData(i-1).cases
                }
                totalToday - totalPrev
            }
        }
        val newDeaths: Int = {
            if (i == 0) d.deaths
            else {
                val totalToday: Int = d.deaths
                val totalPrev: Int = {
                    sortedData(i-1).deaths
                }
                totalToday - totalPrev
            }
        }
        DailySnapshot(d.date, newCases, newDeaths, d.cases, d.deaths)
    })
    
}

defined function totalNewDaily

## The Curve: Total Cases, USA

Just 2020 COVID-19 Cases, cumulative, as reported by the New York Times (https://github.com/nytimes/covid-19-data).

**This is the scary graph!** The cases keep piling up!

In [217]:
val curve_total_cases: Vector[Int] = totalNewDaily(state = None).map(_.totalCases)

val curve_total_cases_trace = Scatter(
  (1 to curve_total_deaths.size),
  curve_total_cases,
  name = "2020 Covid Total Cases",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(0, 204, 0, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(curve_total_cases_trace)

val layout = Layout("Total USA COVID Cases")

plot(data, layout)

curve_total_cases: Vector[Int] = Vector(
  1,
...
curve_total_cases_trace: Scatter = Scatter(
  Some(
...
data: Seq[Scatter] = List(
  Scatter(
...
layout: Layout = Layout(
  Some("Total USA COVID Cases"),
...
res216_4: String = "plot-5d24f685-5e88-4fd6-b3df-9781871e2627"

## The Curve: Total Deaths, USA

Just 2020 COVID-19 Deaths, cumulative as reported by the New York Times (https://github.com/nytimes/covid-19-data).

**This is another scary graph!** The deaths keep piling up!

In [218]:
val curve_total_deaths: Vector[Int] = totalNewDaily(state = None).map(_.totalDeaths)

val curve_total_deaths_trace = Scatter(
  (1 to curve_new_deaths.size),
  curve_total_deaths,
  name = "2020 Covid Death-Toll",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 0, 0, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(curve_total_deaths_trace)

val layout = Layout("Total US Deaths")

plot(data, layout)

curve_total_deaths: Vector[Int] = Vector(
  0,
...
curve_total_deaths_trace: Scatter = Scatter(
  Some(
...
data: Seq[Scatter] = List(
  Scatter(
...
layout: Layout = Layout(
  Some("Total US Deaths"),
...
res217_4: String = "plot-a6db01fb-c198-4cd6-a491-be66f84b67a0"

## The Curve: Total Cases & Deaths, USA

Just 2020 COVID-19 Deaths, cumulative as reported by the New York Times (https://github.com/nytimes/covid-19-data).

Juxatposing the above two graphs.

In [219]:


val data = Seq(curve_total_deaths_trace, curve_total_cases_trace)

val layout = Layout("Total USA COVID Cases and Deaths")

plot(data, layout)

data: Seq[Scatter] = List(
  Scatter(
...
layout: Layout = Layout(
  Some("Total USA COVID Cases and Deaths"),
...
res218_2: String = "plot-811bbb58-cefc-4397-9a4e-c20aaa091838"

## Case-Fatality-Rate: USA

In [220]:
//curve_total_deaths
//curve_total_cases

val cfr: Vector[Double] = curve_total_deaths.zip(curve_total_cases).map( c => {
    c._1.toDouble / c._2.toDouble
})



val case_fatality_rate = Scatter(
  (1 to cfr.size),
  cfr,
  name = "Case Fatality Rate",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 0, 0, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(case_fatality_rate)

val layout = Layout("2020 Covid Case Fatality Rate")

plot(data, layout)

cfr: Vector[Double] = Vector(
  0.0,
...
case_fatality_rate: Scatter = Scatter(
  Some(
...
data: Seq[Scatter] = List(
  Scatter(
...
layout: Layout = Layout(
  Some("2020 Covid Case Fatality Rate"),
...
res219_4: String = "plot-263a1bd7-d0f7-43bb-a5ca-4dd351f522fd"

## “Active” Cases, USA

Simply piling up “cases” is misleading. Patients get over COVID-19, or they die. This graph overlays “total cases” with a graph of cases, with cases over 2 weeks old removed, and deaths removed.

In [221]:
// totalNewDaily

val threshold: Int = 7

val active_cases_usa: Vector[Int] = {
    totalNewDaily().zipWithIndex.map( z => {
        val tnd: DailySnapshot = z._1
        val i: Int = z._2
        if (i < threshold) tnd.totalCases
        else {
            val recovered: Int = totalNewDaily()(i - threshold).totalCases
            val dead: Int = totalNewDaily()(i - threshold).totalDeaths
            tnd.totalCases - (dead + recovered)
        }
    })
}

val curve_active_cases_trace = Scatter(
  (1 to active_cases_usa.size),
  active_cases_usa,
  name = "2020 Covid Active Cases",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(0, 0, 204, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(curve_active_cases_trace, curve_new_cases_trace, curve_total_cases_trace)

val layout = Layout(s"Active Cases: USA (Minus Recovered and Dead): threshold = ${threshold} days.")

plot(data, layout)

threshold: Int = 7
active_cases_usa: Vector[Int] = Vector(
  1,
...
curve_active_cases_trace: Scatter = Scatter(
  Some(
...
data: Seq[Scatter] = List(
  Scatter(
...
layout: Layout = Layout(
  Some("Active Cases: USA (Minus Recovered and Dead): threshold = 7 days."),
...
res220_5: String = "plot-b67c480f-cdf8-469e-ab20-6ddb1354bdb9"

## New Cases vs. Total Cases

This juxtaposition is informative.

In [222]:

val new_cases_usa: Vector[Int] = {
    totalNewDaily().map( z => z.newCases)
}

val curve_new_cases_trace = Scatter(
  (1 to new_cases_usa.size),
  new_cases_usa,
  name = "2020 Covid New Cases, Daily",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 0, 0, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(curve_total_cases_trace, curve_new_cases_trace)

val layout = Layout(s"New Cases vs. Total Cases: USA")

plot(data, layout)

new_cases_usa: Vector[Int] = Vector(
  1,
...
curve_new_cases_trace: Scatter = Scatter(
  Some(
...
data: Seq[Scatter] = List(
  Scatter(
...
layout: Layout = Layout(
  Some("New Cases vs. Total Cases: USA"),
...
res221_4: String = "plot-e24fc8dd-7a7d-474b-8deb-3207002d3394"

## Recovered Cases, USA



In [223]:
// totalNewDaily


val recovered_cases_usa: Vector[Int] = {
    totalNewDaily().zipWithIndex.map( z => {
        val tnd: DailySnapshot = z._1
        val i: Int = z._2
        if (i < threshold) 0
        else {
            val recovered: Int = totalNewDaily()(i - threshold).totalCases
            val dead: Int = totalNewDaily()(i - (threshold/2)).totalDeaths
            recovered - dead
        }
     })
}

val curve_recovered_cases_trace = Scatter(
  (1 to recovered_cases_usa.size),
  recovered_cases_usa,
  name = "2020 Covid Recovered Cases",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 0, 0, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(curve_recovered_cases_trace, curve_total_cases_trace)

val layout = Layout(s"Recovered Cases & Total Cases: threshold = ${threshold} days.")

plot(data, layout)

recovered_cases_usa: Vector[Int] = Vector(
  0,
...
curve_recovered_cases_trace: Scatter = Scatter(
  Some(
...
data: Seq[Scatter] = List(
  Scatter(
...
layout: Layout = Layout(
  Some("Recovered Cases & Total Cases: threshold = 7 days."),
...
res222_4: String = "plot-e3501e2d-95b4-4bf3-843a-03757a014f89"

## Compared to the Flu

Throughout a year like 2017, the death-toll for Influenzas and Pneumonias mounts, day to day. Here we chart that death-toll, assuming the annual deaths are evenly distributed across the year. 

But the flu-season is, as defined by the CDC, 6 months, from October through March. So we can also compare daily flu deaths and their mounting toll, over a six-month season.

Source for USA Data: [“Deaths and Mortality”, CDC](https://www.cdc.gov/nchs/fastats/deaths.htm).

In [224]:


val usa_2017_mounting_flu_toll: Vector[Int] = {
    val daily = usa_2017_Total_Deaths_IPn / 365
    (1 to curve_total_deaths.size).toVector.map( i => {
        if (i == 1) daily
        else daily + (daily * (i - 1))
    })
}

val usa_2017_mounting_flu_toll_6: Vector[Int] = {
    val daily = usa_2017_Total_Deaths_IPn / 365 * 2
    (1 to curve_total_deaths.size).toVector.map( i => {
        if (i == 1) daily
        else daily + (daily * (i - 1))
    })
}

val usa_2017_Daily_Deaths_IPn_trace = Scatter(
  (1 to curve_total_deaths.size),
  usa_2017_mounting_flu_toll,
  name = "2017 Influenza Death Toll",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 204, 0, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val usa_2017_Daily_Deaths_IPn_6mo_trace = Scatter(
  (1 to curve_total_deaths.size),
  usa_2017_mounting_flu_toll_6,
  name = "2017 Influenza Death Toll (6 mo. season)",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(0, 204, 204, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(curve_total_deaths_trace, usa_2017_Daily_Deaths_IPn_trace, usa_2017_Daily_Deaths_IPn_6mo_trace)

val layout = Layout("COVID Death Toll vs. Flu Season, 2017")

plot(data, layout)

usa_2017_mounting_flu_toll: Vector[Int] = Vector(
  152,
...
usa_2017_mounting_flu_toll_6: Vector[Int] = Vector(
  304,
...
usa_2017_Daily_Deaths_IPn_trace: Scatter = Scatter(
  Some(
...
usa_2017_Daily_Deaths_IPn_6mo_trace: Scatter = Scatter(
  Some(
...
data: Seq[Scatter] = List(
  Scatter(
...
layout: Layout = Layout(
  Some("COVID Death Toll vs. Flu Season, 2017"),
...
res223_6: String = "plot-de92e4eb-325c-45af-8fc0-c7e3255d5d83"

## Are More People Dying?

For England and Wales, we have week-by-week data for deaths (from all causes), in 2020, through the week of March 27, and an average of deaths in each of the first 13 weeks in 2015-2019.

Source for Data:[The Office of National Statistics, UK](https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/datasets/weeklyprovisionalfiguresondeathsregisteredinenglandandwales)

In [226]:
/*
val uk_weeks = Vector("3-Jan-20","10-Jan-20","17-Jan-20","24-Jan-20","31-Jan-20","7-Feb-20","14-Feb-20","21-Feb-20","28-Feb-20","6-Mar-20","13-Mar-20","20-Mar-20","27-Mar-20")
val uk_Death = Vector(12254,14058,12990,11856,11612,10986,10944,10841,10816,10895,11019,10645,11141)
val uk_Deaths_5yr_Ave = Vector(12175,13822,13216,12760,12206,11925,11627,11548,11183,11498,11205,10573,10130)
val uk_2018_Resp_Deaths = Vector(2361,3075,2829,2672,2479,2197,2081,2128,1900,2302,2271,1946,1657)
*/


val cumuUkDeathsByWeek2020: Vector[Int] = {
    uk_Death.zipWithIndex.map( t => {
        val d = t._1
        val i = t._2
        if (i == 0 ) {
            d
        }
        else {
            d + (uk_Death.take(i+1).sum)
        }
    })
}

val cumuUkDeathsByWeekAve: Vector[Int] = {
    uk_Deaths_5yr_Ave.zipWithIndex.map( t => {
        val d = t._1
        val i = t._2
        if (i == 0 ) {
            d
        }
        else {
            d + (uk_Deaths_5yr_Ave.take(i+1).sum)
        }
    })
}

val trace_uk_death_2020 = Scatter(
  (1 to cumuUkDeathsByWeek2020.size).toSeq,
  cumuUkDeathsByWeek2020.toSeq,
  fill = Fill.ToZeroY, 
  name = "2020"
)

val trace_uk_death_5yr_ave = Scatter(
  (1 to cumuUkDeathsByWeek2020.size).toSeq,
  cumuUkDeathsByWeekAve.toSeq,
  fill = Fill.ToNextY,
  name = "Average (2015-2019)"
)

val data = Seq(trace_uk_death_2020, trace_uk_death_5yr_ave)

val layout = Layout("England and Wales: Weekly Death Toll, all causes")

plot(data, layout)

cumuUkDeathsByWeek2020: Vector[Int] = Vector(
  12254,
...
cumuUkDeathsByWeekAve: Vector[Int] = Vector(
  12175,
...
trace_uk_death_2020: Scatter = Scatter(
  Some(
...
trace_uk_death_5yr_ave: Scatter = Scatter(
  Some(
...
data: Seq[Scatter] = List(
  Scatter(
...
layout: Layout = Layout(
  Some("England and Wales: Weekly Death Toll, all causes"),
...
res225_6: String = "plot-b0b09e58-f67e-4fd1-b628-d3de4f57fdf4"